In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pylab as plt
import xmltodict, json
import requests
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/sebscho/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# adding in my api key 
with open('zillow_key.txt', 'r') as f:
     myAPI = f.readline()

In [ ]:
# adding params as listed in api documentation: https://www.zillow.com/howto/api/GetDeepComps.htm
params = {'zws-id': myAPI,
         'address': '220 East 73rd Street',
         'citystatezip': '10021',
         'rentzestimate': 'True'}
# test
result = requests.get('http://www.zillow.com/webservice/GetDeepSearchResults.htm', params = params)

In [ ]:
# see first X values 
result.text[100:]

In [ ]:
d = xmltodict.parse(result.text)

In [ ]:
d

In [ ]:
# correct place to parse for building address
address = d['SearchResults:searchresults']['request']['address']

In [ ]:
# results provides multiple listing for the same address. 
# Bedrooms are nested here

bedrooms = d['SearchResults:searchresults']['response']['results']['result'][0]['bedrooms']
bathrooms = d['SearchResults:searchresults']['response']['results']['result'][0]['bathrooms']

print ('This apartment at {} has {} bedrooms and {} bathrooms'.format(address, bedrooms, bathrooms))                                                                                      

In [ ]:
# In each response, you can extract the date sold and price for each sale
date_sold = d['SearchResults:searchresults']['response']['results']['result'][2]['lastSoldDate']
price = d['SearchResults:searchresults']['response']['results']['result'][2]['lastSoldPrice']['#text']                                                                                      
currency = d['SearchResults:searchresults']['response']['results']['result'][2]['lastSoldPrice']['@currency']

print ('Sold on {} for {}{}'.format(date_sold,price,currency))                                                                                      

In [ ]:
# within each 'result,' the first position indcludes info about the unit (above)
# each successive item is a record for each time its been sold, but doesn't seem to be for different units (just any unit at that address)
d['SearchResults:searchresults']['response']['results']['result'][1].keys()

In [ ]:
# building type (ie cooperative, )
building_type = d['SearchResults:searchresults']['response']['results']['result'][0]['useCode']

# finished building sq ft - not at unit level
building_sqft = d['SearchResults:searchresults']['response']['results']['result'][0]['finishedSqFt']

# lot square feet - only one per result. also not at unit level
lot_sqft = d['SearchResults:searchresults']['response']['results']['result'][0]['lotSizeSqFt']

___

In [ ]:
# do for other apt with unit
params = {'zws-id': myAPI,
         'address': '1735 York Ave',
         'citystatezip': '10128',
         'rentzestimate': 'True'}
york = requests.get('http://www.zillow.com/webservice/GetDeepSearchResults.htm', params = params)


In [ ]:
# testing another
york = xmltodict.parse(york.text)

In [ ]:
# testing another apt in a dumb way
address = york['SearchResults:searchresults']['request']['address']
bedrooms = york['SearchResults:searchresults']['response']['results']['result'][0]['bedrooms']
bathrooms = york['SearchResults:searchresults']['response']['results']['result'][0]['bathrooms']
# date_sold = york['SearchResults:searchresults']['response']['results']['result'][2]['lastSoldDate']
# price = york['SearchResults:searchresults']['response']['results']['result'][2]['lastSoldPrice']['#text']                                                                                      
# currency = york['SearchResults:searchresults']['response']['results']['result'][2]['lastSoldPrice']['@currency']
building_type = york['SearchResults:searchresults']['response']['results']['result'][0]['useCode']


In [ ]:
print ('This Yorkville {} at {} has {} bedroom(s) and {} bathroom(s)'.format(building_type,address,bedrooms,bathrooms))                                                                                      

___

In [ ]:
# d = {'result': {'list': {'one_more_annoying_thing': ['a', 'b', 'c']}}}
# # step 1 -- you should iteratively run the next few lines to see the outputs come up
# # as you go
# d.keys()
# # ok, "result" is some key we can just skip through

# # so then we just filter in to "result" and look at the keys of that dict
# d["result"].keys()
# # and after running this one, we see 'list' so let's just keep filtering down
# d["result"]["list"].keys()

# # and again...
# d["result"]["list"]["one_more_annoying_thing"].keys()
# #aha! this line raises an exception about a "list" not having "keys"
# # so we know we've found a list that we probably care about
# # in your case, this may be a string object instead of a list, depending
# # on the output format

# # after running this, 
# filtered = d["result"]["list"]["one_more_annoying_thing"]
# a = filtered[0]
# b = filtered[1]

In [79]:
# do for other apt with unit
params = {'zws-id': myAPI,
         'address': '220 E 73rd St # 1D',
         'citystatezip': '10128',
         'rentzestimate': 'True'}
test = requests.get('http://www.zillow.com/webservice/GetDeepSearchResults.htm', params = params)


In [80]:
l = xmltodict.parse(test.text)

In [81]:
# testing another apt in a dumb way
address = l['SearchResults:searchresults']['request']['address']

address

'220 E 73rd St # 1D'

In [83]:
l['SearchResults:searchresults']['response']['results']['result']['bathrooms']
# york['SearchResults:searchresults']['response']['results']['result'][0]['bedrooms']


'1.0'

In [89]:
addr_str = '''
781 5 AVENUE # 30T,10022
781 5 AVENUE # 30T,10022
650 PARK AVENUE # 9D,10065
650 PARK AVENUE # 9D,10065
3 EAST 71 STREET # 3D,10021
3 EAST 71 STREET # 3D,10021
962 5 AVENUE # 8C,10075
962 5 AVENUE # 8C,10075
888 PARK AVENUE # PHA,10075
888 PARK AVENUE # PHA,10075
118 EAST 60 STREET # 20B,10022
118 EAST 60 STREET # 20B,10022
773 LEXINGTON AVENUE # 9D,10065
773 LEXINGTON AVENUE # 9D,10065
571 PARK AVENUE # 1507,10065
571 PARK AVENUE # 1507,10065
125 EAST 63 STREET # 6C,10065
125 EAST 63 STREET # 6C,10065
1250 3 AVENUE # 10D,10021
1250 3 AVENUE # 10D,10021
177 EAST 74 STREET # 15E,10021
177 EAST 74 STREET # 15E,10021
111 EAST 75 STREET # 4A,10021
111 EAST 75 STREET # 4A,10021
151 EAST 76 STREET # 8H,10021
151 EAST 76 STREET # 8H,10021
1091 LEXINGTON AVENUE # 5F,10075
1091 LEXINGTON AVENUE # 5F,10075
1071 3 AVENUE # 14H,10065
1071 3 AVENUE # 14H,10065
1131 3 AVENUE # 4F,10065
1131 3 AVENUE # 4F,10065
220 EAST 67 STREET # 3E,10065
220 EAST 67 STREET # 3E,10065
245 EAST 72 STREET # 9F,10021
245 EAST 72 STREET # 9F,10021
210 EAST 73 STREET # 5E,10021
210 EAST 73 STREET # 5F,10021
210 EAST 73 STREET # 5E,10021
210 EAST 73 STREET # 5F,10021
1285 3 AVENUE # 4C,10021
1285 3 AVENUE # 4C,10021
225 EAST 74 STREET # 4B,10021
225 EAST 74 STREET # 4B,10021
225 EAST 76 STREET # 6B,10021
225 EAST 76 STREET # 6B,10021
315 EAST 65 STREET # PHA,10065
315 EAST 65 STREET # PHA,10065
1325 1 AVENUE # B1005,10021
1325 1 AVENUE # B1005,10021
353 EAST 72 STREET # 25B,10021
353 EAST 72 STREET # 25B,10021
345 EAST 73 STREET # 11J,10021
345 EAST 73 STREET # 11J,10021
1461 1 AVENUE # 19D,10075
1461 1 AVENUE # 2H,10075
1461 1 AVENUE # 3J,10075
1461 1 AVENUE # 2H,10075
1461 1 AVENUE # 3J,10075
1461 1 AVENUE # 19D,10075
435 EAST 65 STREET # 14B,10065
435 EAST 65 STREET # 14B,10065
1344 1 AVENUE # 5J,10021
1344 1 AVENUE # 5J,10021
440 EAST 79 STREET # 5L,10075
440 EAST 79 STREET # 5L,10075
520 EAST 72 STREET # 2T,10021
520 EAST 72 STREET # 2T,10021
121 EAST 88 STREET # 1B,10128
121 EAST 88 STREET # 1B,10128
218 EAST 82 STREET # 2RE,10028
218 EAST 82 STREET # 2RE,10028
204 EAST 90 STREET # 1E,10128
204 EAST 90 STREET # 4W,10128
204 EAST 90 STREET # 4W,10128
204 EAST 90 STREET # 1E,10128
950 PARK AVENUE # 5A,10028
950 PARK AVENUE # 5A,10028
1225 MADISON AVENUE # 3B,10128
1225 MADISON AVENUE # 3B,10128
48 EAST 89 STREET # 18B,10128
48 EAST 89 STREET # 18B,10128
1254 MADISON AVENUE # 3C,10128
1254 MADISON AVENUE # 3C,10128
1274 MADISON AVENUE # 14B,10128
1274 MADISON AVENUE # 14B,10128
1160 PARK AVENUE # 3D,10128
1160 PARK AVENUE # 3D,10128
173 EAST 79 STREET # 15C,10075
173 EAST 79 STREET # 7C,10075
173 EAST 79 STREET # 15C,10075
173 EAST 79 STREET # 7C,10075
179 EAST 79 STREET # 5A,10075
179 EAST 79 STREET # 5A,10075
163 EAST 81 STREET # 3D,10028
163 EAST 81 STREET # 3D,10028
108 EAST 82 STREET # 8B,10028
108 EAST 82 STREET # 8B,10028
103 EAST 84 STREET # PHD,10028
103 EAST 84 STREET # PHD,10028
108 EAST 86 STREET # 14S,10028
108 EAST 86 STREET # 14S,10028
1095 PARK AVENUE # 16C,10128
1095 PARK AVENUE # 16C,10128
108 EAST 91 STREET # 9D,10128
108 EAST 91 STREET # 9D,10128
175 EAST 93 STREET # 4D,10128
175 EAST 93 STREET # 4D,10128
130 EAST 94 STREET # 1E,10128
130 EAST 94 STREET # 1E,10128
1681 2 AVENUE # 17A,10128
1681 2 AVENUE # 17A,10128
1589 3 AVENUE # 7A,10128
1589 3 AVENUE # 7H,10128
1589 3 AVENUE # 7H,10128
1589 3 AVENUE # 7A,10128
333 EAST 79 STREET # 4L,10075
333 EAST 79 STREET # 4L,10075
1682 2 AVENUE # 7A,10128
1682 2 AVENUE # 7A,10128
311 EAST 87 STREET # 4V,10128
311 EAST 87 STREET # 4V,10128
321 EAST 89 STREET # 1D,10128
321 EAST 89 STREET # 1D,10128
1761 1 AVENUE # 24A,10128
1761 1 AVENUE # 24A,10128
1781 1 AVENUE # 17E,10128
1781 1 AVENUE # 17E,10128
345 EAST 93 STREET # 4K,10128
345 EAST 93 STREET # 11J,10128
345 EAST 93 STREET # 4K,10128
345 EAST 93 STREET # 11J,10128
436 EAST 86 STREET # 30J,10028
436 EAST 86 STREET # 30J,10028
1652 1 AVENUE # 8K,10028
1652 1 AVENUE # 8K,10028
425 EAST 86 STREET # 11BC,10028
425 EAST 86 STREET # 11BC,10028
1725 YORK AVENUE # 15G,10128
1725 YORK AVENUE # 15G,10128
10 EAST END AVENUE # 10F,10075
10 EAST END AVENUE # 10F,10075
505 EAST 82 STREET # 4E,10028
505 EAST 82 STREET # 6D,10028
505 EAST 82 STREET # 4E,10028
505 EAST 82 STREET # 6D,10028
535 EAST 86 STREET # 10J,10028
535 EAST 86 STREET # 10J,10028
'''

addresses = addr_str.split('\n') # you get back  ["a", "b", "c"]
addresses = addresses[1:]


['781 5 AVENUE # 30T,10022',
 '781 5 AVENUE # 30T,10022',
 '650 PARK AVENUE # 9D,10065',
 '650 PARK AVENUE # 9D,10065',
 '3 EAST 71 STREET # 3D,10021',
 '3 EAST 71 STREET # 3D,10021',
 '962 5 AVENUE # 8C,10075',
 '962 5 AVENUE # 8C,10075',
 '888 PARK AVENUE # PHA,10075',
 '888 PARK AVENUE # PHA,10075',
 '118 EAST 60 STREET # 20B,10022',
 '118 EAST 60 STREET # 20B,10022',
 '773 LEXINGTON AVENUE # 9D,10065',
 '773 LEXINGTON AVENUE # 9D,10065',
 '571 PARK AVENUE # 1507,10065',
 '571 PARK AVENUE # 1507,10065',
 '125 EAST 63 STREET # 6C,10065',
 '125 EAST 63 STREET # 6C,10065',
 '1250 3 AVENUE # 10D,10021',
 '1250 3 AVENUE # 10D,10021',
 '177 EAST 74 STREET # 15E,10021',
 '177 EAST 74 STREET # 15E,10021',
 '111 EAST 75 STREET # 4A,10021',
 '111 EAST 75 STREET # 4A,10021',
 '151 EAST 76 STREET # 8H,10021',
 '151 EAST 76 STREET # 8H,10021',
 '1091 LEXINGTON AVENUE # 5F,10075',
 '1091 LEXINGTON AVENUE # 5F,10075',
 '1071 3 AVENUE # 14H,10065',
 '1071 3 AVENUE # 14H,10065',
 '1131 3 AVENUE # 4F

In [101]:
import time

errors = []
final = []

for addr_zip in addresses:
    addr, zips = addr_zip.split(",")
    # delays for 2 seconds
    time.sleep(2) 
    # looping all addresses through api 
    params = {'zws-id': myAPI,
             'address': addr,
             'citystatezip': zips,
             'rentzestimate': 'True'}
    resp = requests.get('http://www.zillow.com/webservice/GetDeepSearchResults.htm', params = params)
    
    if "no exact match found for input address" in (resp.text):
        errors.append(addr_zip)
        continue
   
    # parse each resp, and name it d 
    m = xmltodict.parse(resp.text)
    
    
    # pull out bedrooms for each d
    results = m['SearchResults:searchresults']['response']['results']['result']
    
    if 'bedrooms' in results:
        bedrooms = m['SearchResults:searchresults']['response']['results']['result']['bedrooms']
    else:
        errors.append(addr_zip)
        continue 
                
    # add the address and bedroom num to "final" list
    final.append((addr, bedrooms))
    
    

ValueError: not enough values to unpack (expected 2, got 1)

In [110]:
print ('\n'.join(np.unique(final)))


0
1
108 EAST 82 STREET # 8B
108 EAST 91 STREET # 9D
1095 PARK AVENUE # 16C
111 EAST 75 STREET # 4A
1160 PARK AVENUE # 3D
118 EAST 60 STREET # 20B
121 EAST 88 STREET # 1B
130 EAST 94 STREET # 1E
1682 2 AVENUE # 7A
1725 YORK AVENUE # 15G
173 EAST 79 STREET # 15C
175 EAST 93 STREET # 4D
1781 1 AVENUE # 17E
179 EAST 79 STREET # 5A
2
204 EAST 90 STREET # 4W
210 EAST 73 STREET # 5E
210 EAST 73 STREET # 5F
218 EAST 82 STREET # 2RE
220 EAST 67 STREET # 3E
225 EAST 76 STREET # 6B
245 EAST 72 STREET # 9F
3
315 EAST 65 STREET # PHA
321 EAST 89 STREET # 1D
345 EAST 73 STREET # 11J
345 EAST 93 STREET # 11J
345 EAST 93 STREET # 4K
4
425 EAST 86 STREET # 11BC
435 EAST 65 STREET # 14B
505 EAST 82 STREET # 4E
535 EAST 86 STREET # 10J
650 PARK AVENUE # 9D
888 PARK AVENUE # PHA


In [109]:
print ('\n'.join(np.unique(errors)))

10 EAST END AVENUE # 10F,10075
103 EAST 84 STREET # PHD,10028
1071 3 AVENUE # 14H,10065
108 EAST 86 STREET # 14S,10028
1091 LEXINGTON AVENUE # 5F,10075
1131 3 AVENUE # 4F,10065
1225 MADISON AVENUE # 3B,10128
125 EAST 63 STREET # 6C,10065
1250 3 AVENUE # 10D,10021
1254 MADISON AVENUE # 3C,10128
1274 MADISON AVENUE # 14B,10128
1285 3 AVENUE # 4C,10021
1325 1 AVENUE # B1005,10021
1344 1 AVENUE # 5J,10021
1461 1 AVENUE # 19D,10075
1461 1 AVENUE # 2H,10075
1461 1 AVENUE # 3J,10075
151 EAST 76 STREET # 8H,10021
1589 3 AVENUE # 7A,10128
1589 3 AVENUE # 7H,10128
163 EAST 81 STREET # 3D,10028
1652 1 AVENUE # 8K,10028
1681 2 AVENUE # 17A,10128
173 EAST 79 STREET # 7C,10075
1761 1 AVENUE # 24A,10128
177 EAST 74 STREET # 15E,10021
204 EAST 90 STREET # 1E,10128
225 EAST 74 STREET # 4B,10021
3 EAST 71 STREET # 3D,10021
311 EAST 87 STREET # 4V,10128
333 EAST 79 STREET # 4L,10075
353 EAST 72 STREET # 25B,10021
436 EAST 86 STREET # 30J,10028
440 EAST 79 STREET # 5L,10075
48 EAST 89 STREET # 18B,10128
5

In [77]:
m['SearchResults:searchresults']['response']['results']['result'].keys()

odict_keys(['zpid', 'links', 'address', 'FIPScounty', 'useCode', 'yearBuilt', 'zestimate', 'rentzestimate', 'localRealEstate'])